In [1]:
import json
from glob import glob
from multiprocessing import Pool
from pathlib import Path
import os
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from stexls.util import file_watcher
from pickle import dumps, loads, dump, load

from stexls.util.latex import parser

from stexls import *

%cd ~/Downloads

/home/marian/Downloads


In [2]:
def parse_files(root='smglom', limit: int = -1):
    ww = file_watcher.WorkspaceWatcher(root, filter='*.tex')
    changes = ww.update()
    with Pool() as pool:
        parsed_files = list(tqdm(pool.imap_unordered(parse, list(changes.created | changes.modified)[:limit])))
    return parsed_files

use_cache = True
cachefile = Path('/tmp/cache.bin')
if cachefile.is_file() and use_cache:
    with open(cachefile, 'rb') as fd:
        parsed_files = load(fd)
else:
    parsed_files = parse_files()
    with open(cachefile, 'w+b') as fd:
        dump(parsed_files, fd)

In [3]:
parsed_files[0].defis[0].name

'Pythagorean-triple'

In [4]:
for pf in parsed_files:
    if pf.errors:
        print(pf.errors)
        print()
        print('-'*80)
        print()

defaultdict(<class 'list'>, {[Location uri="smglom/algebra/source/algebraic-structure.de.tex" range=[Range (7 11) (7 77)]]: [CompilerException('Defi argument count mismatch: Expected 2 vs actual 3.')]})

--------------------------------------------------------------------------------

defaultdict(<class 'list'>, {[Location uri="smglom/theresas-playground/source/poisson-equation.tex" range=[Range (1 3) (1 58)]]: [CompilerException('ImportModule "path" argument not supported.')]})

--------------------------------------------------------------------------------

defaultdict(<class 'list'>, {[Location uri="smglom/theresas-playground/lib/pre.ro.tex" range=[Range (0 0) (6 17)]]: [LatexException('Broken parser stack: [[Node "documentclass[mh]{smglom}\n\\input{localpaths}\n\\usepackage[utf8]{inputenc}\n\\usepackage[romanian]{babel}\n\\input{\\MathHub{smglom/theresas-playground/lib/preamble}}\n\\begin{document}"], [Environment name=[Node "{document}"]]]')]})

----------------------------------

In [5]:
with Pool() as pool:
    compiled_files = list(map(StexObject.compile, tqdm(parsed_files)))

In [6]:
ok = list(filter(None, compiled_files))

In [10]:
parsed_files[-1].__dict__

{'path': PosixPath('smglom/algebra/source/ring.tex'),
 'modsigs': [[Modsig name=modsig]],
 'modnls': [],
 'modules': [],
 'trefis': [],
 'defis': [],
 'syms': [[Sym* i=1 tokens=[ring]],
  [Sym i=2 tokens=[additive, structure]],
  [Sym i=2 tokens=[multiplicative, structure]],
  [Sym i=1 tokens=[multiplication]],
  [Sym i=1 tokens=[addition]],
  [Sym i=2 tokens=[additive, inverse]]],
 'symdefs': [[Symdef name=additive-inverse],
  [Symdef name=additive-inverse],
  [Symdef name=zero],
  [Symdef name=subtraction],
  [Symdef name=subtraction],
  [Symdef name=one]],
 'importmodules': [],
 'gimports': [[public gimport ringoid from "/home/marian/Downloads/smglom/algebra/source/ringoid.tex"],
  [public gimport fundefeq from "/home/marian/Downloads/smglom/sets/source/fundefeq.tex"]],
 'errors': defaultdict(list, {})}

In [7]:
for o in ok:
    try:
        print(o.format())
        print()
        print('-'*80)
        print()
    except:
        print(o.__dict__)
        raise

Contains files:
	smglom/primes/source/pythagoreantriple.en.tex

Depends on:
	/home/marian/Downloads/smglom/primes/source/pythagoreantriple.tex

Symbols: <no symbols>

References:
	smglom/primes/source/pythagoreantriple.en.tex:1:33:pythagoreantriple
	smglom/primes/source/pythagoreantriple.en.tex:3:6:pythagoreantriple.Pythagorean-triple
	smglom/primes/source/pythagoreantriple.en.tex:5:66:pythagoreantriple.primitive
	smglom/primes/source/pythagoreantriple.en.tex:4:12:naturalnumbers
	smglom/primes/source/pythagoreantriple.en.tex:4:4:naturalnumbers.natural-number
	smglom/primes/source/pythagoreantriple.en.tex:6:4:pythagoreantriple.Pythagorean-triple
	smglom/primes/source/pythagoreantriple.en.tex:7:10:coprime
	smglom/primes/source/pythagoreantriple.en.tex:7:4:coprime.coprime

Errors: <no errors>

--------------------------------------------------------------------------------

Contains files:
	smglom/primes/source/mersenneprime.de.tex

Depends on:
	/home/marian/Downloads/smglom/primes/source


Contains files:
	smglom/graphs/source/graphspanningsubgraph.tex

Depends on:
	/home/marian/Downloads/smglom/graphs/source/graphinducedsubgraph.tex

Symbols:
	smglom/graphs/source/graphspanningsubgraph.tex:1:2:[public Symbol modsig/MODULE]
	smglom/graphs/source/graphspanningsubgraph.tex:3:4:[public Symbol modsig.spanning-subgraph/SYMBOL]

References:
	smglom/graphs/source/graphspanningsubgraph.tex:2:4:graphinducedsubgraph

Errors:
	smglom/graphs/source/graphspanningsubgraph.tex:1:2:Invalid modsig filename: Expected "modsig.tex"

--------------------------------------------------------------------------------

Contains files:
	smglom/sets/source/equinumerous.de.tex

Depends on:
	/home/marian/Downloads/smglom/sets/source/equinumerous.tex

Symbols: <no symbols>

References:
	smglom/sets/source/equinumerous.de.tex:1:31:equinumerous
	smglom/sets/source/equinumerous.de.tex:4:6:equinumerous.equinumerous
	smglom/sets/source/equinumerous.de.tex:3:18:set
	smglom/sets/source/equinumerous.de.tex:3

	smglom/constants/source/nplex.tex:1:2:Invalid modsig filename: Expected "modsig.tex"

--------------------------------------------------------------------------------

Contains files:
	smglom/algebra/source/generating-set-group.en.tex

Depends on:
	/home/marian/Downloads/smglom/algebra/source/generating-set-group.tex

Symbols: <no symbols>

References:
	smglom/algebra/source/generating-set-group.en.tex:1:33:generating-set-group
	smglom/algebra/source/generating-set-group.en.tex:3:79:generating-set-group.generated
	smglom/algebra/source/generating-set-group.en.tex:5:4:generating-set-group.generating-set
	smglom/algebra/source/generating-set-group.en.tex:10:4:generating-set-group.generated-subgroup
	smglom/algebra/source/generating-set-group.en.tex:14:57:generating-set-group.generates
	smglom/algebra/source/generating-set-group.en.tex:15:19:generating-set-group.generator
	smglom/algebra/source/generating-set-group.en.tex:15:40:generating-set-group.generator
	smglom/algebra/source/genera


--------------------------------------------------------------------------------

Contains files:
	smglom/numbers/source/factorion.en.tex

Depends on:
	/home/marian/Downloads/smglom/numbers/source/factorion.tex

Symbols: <no symbols>

References:
	smglom/numbers/source/factorion.en.tex:1:33:factorion
	smglom/numbers/source/factorion.en.tex:3:6:factorion.factorion
	smglom/numbers/source/factorion.en.tex:3:35:naturalnumbers
	smglom/numbers/source/factorion.en.tex:3:28:naturalnumbers.natural-number
	smglom/numbers/source/factorion.en.tex:4:14:factorial
	smglom/numbers/source/factorion.en.tex:4:8:factorial.factorial
	smglom/numbers/source/factorion.en.tex:5:11:positional-number-system
	smglom/numbers/source/factorion.en.tex:5:4:positional-number-system.digit

Errors: <no errors>

--------------------------------------------------------------------------------

Contains files:
	smglom/sets/source/directedSet.zhs.tex

Depends on:
	/home/marian/Downloads/smglom/sets/source/directedSet.tex

S


--------------------------------------------------------------------------------

Contains files:
	smglom/numthyfun/source/summangoldtfunction.tex

Depends on:
	/home/marian/Downloads/smglom/numthyfun/source/mangoldtfunction.tex

Symbols:
	smglom/numthyfun/source/summangoldtfunction.tex:1:2:[public Symbol modsig/MODULE]
	smglom/numthyfun/source/summangoldtfunction.tex:6:2:[public Symbol modsig.summatory-Mangoldt-function/SYMBOL]
	smglom/numthyfun/source/summangoldtfunction.tex:4:2:[public Symbol modsig.summatory-Mangoldt-function/SYMBOL]

References:
	smglom/numthyfun/source/summangoldtfunction.tex:2:2:mangoldtfunction

Errors:
	smglom/numthyfun/source/summangoldtfunction.tex:1:2:Invalid modsig filename: Expected "modsig.tex"

--------------------------------------------------------------------------------

Contains files:
	smglom/numbers/source/multiperfectnumber.en.tex

Depends on:
	/home/marian/Downloads/smglom/numbers/source/multiperfectnumber.tex

Symbols: <no symbols>

Reference

Contains files:
	smglom/graphs/source/empty-graph.tex

Depends on:
	/home/marian/Downloads/smglom/graphs/source/graph.tex
	/home/marian/Downloads/smglom/sets/source/emptyset.tex

Symbols:
	smglom/graphs/source/empty-graph.tex:1:2:[public Symbol modsig/MODULE]
	smglom/graphs/source/empty-graph.tex:4:4:[public Symbol modsig.empty/SYMBOL]
	smglom/graphs/source/empty-graph.tex:5:4:[public Symbol modsig.edgeless/SYMBOL]
	smglom/graphs/source/empty-graph.tex:6:4:[public Symbol modsig.empty-graph/SYMBOL]

References:
	smglom/graphs/source/empty-graph.tex:2:4:graph
	smglom/graphs/source/empty-graph.tex:3:4:emptyset

Errors:
	smglom/graphs/source/empty-graph.tex:1:2:Invalid modsig filename: Expected "modsig.tex"

--------------------------------------------------------------------------------

Contains files:
	smglom/algebra/source/period.zhs.tex

Depends on:
	/home/marian/Downloads/smglom/algebra/source/period.tex

Symbols: <no symbols>

References:
	smglom/algebra/source/period.zhs.tex:1:30:p

--------------------------------------------------------------------------------

Contains files:
	smglom/measure-theory/source/measure.tex

Depends on:
	/home/marian/Downloads/smglom/measure-theory/source/sigma-additive.tex

Symbols:
	smglom/measure-theory/source/measure.tex:1:2:[public Symbol modsig/MODULE]
	smglom/measure-theory/source/measure.tex:3:4:[public Symbol modsig.measure/SYMBOL]
	smglom/measure-theory/source/measure.tex:4:4:[public Symbol modsig.measure-space/SYMBOL]
	smglom/measure-theory/source/measure.tex:5:4:[public Symbol modsig.finite/SYMBOL]

References:
	smglom/measure-theory/source/measure.tex:2:4:sigma-additive

Errors:
	smglom/measure-theory/source/measure.tex:1:2:Invalid modsig filename: Expected "modsig.tex"

--------------------------------------------------------------------------------

Contains files:
	smglom/primes/source/sexyprime.de.tex

Depends on:
	/home/marian/Downloads/smglom/primes/source/sexyprime.tex

Symbols: <no symbols>

References:
	smglom/pr